<a href="https://colab.research.google.com/github/haraldriisager/ML-Project/blob/evan/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown
import gdown
import pandas as pd

file_id = '1Uf3q-CoSVK84kogmfTAtvIZ3k8crjN1d'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'data.csv', quiet=False)
df = pd.read_csv('data.csv')

print(df.head())

In [ ]:
# Define delay classification based on DelayMinutes
def classify_delay(delay_minutes):
    if delay_minutes <= 15:
        return 'No delay'
    elif 15 < delay_minutes <= 30:
        return 'Slight delay'
    elif 30 < delay_minutes <= 60:
        return 'Moderate delay'
    else:
        return 'Severe delay'

# Apply classification function
df['delay_category'] = df['DelayMinutes'].apply(classify_delay)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Select relevant features
features = ['Airline', 'FlightNumber', 'Origin', 'Destination', 'ScheduledDeparture',
            'ScheduledArrival', 'Distance', 'DelayReason', 'AircraftType']
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
df = df[features + ['delay_category']].copy()

# Encode categorical columns
label_encoder = LabelEncoder()
for col in ['Airline', 'Origin', 'Destination', 'DelayReason', 'AircraftType']:
    df.loc[:, col] = label_encoder.fit_transform(df[col].fillna("Unknown"))

# Convert ScheduledDeparture and ScheduledArrival to datetime
df.loc[:, 'ScheduledDeparture'] = pd.to_datetime(df['ScheduledDeparture'])
df.loc[:, 'ScheduledArrival'] = pd.to_datetime(df['ScheduledArrival'])

# Feature engineering for departure time
df.loc[:, 'DepartureHour'] = df['ScheduledDeparture'].dt.hour
df.loc[:, 'ArrivalHour'] = df['ScheduledArrival'].dt.hour

# Standardize numerical columns
scaler = StandardScaler()
df[['FlightNumber', 'Distance', 'DepartureHour', 'ArrivalHour']] = scaler.fit_transform(
    df[['FlightNumber', 'Distance', 'DepartureHour', 'ArrivalHour']]
)

In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop('delay_category', axis=1)
y = df['delay_category']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Drop datetime columns if they are still in X_train
X_train = X_train.drop(columns=['ScheduledDeparture', 'ScheduledArrival'], errors='ignore')

# Verify all columns are numeric
print(X_train.dtypes)

rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

X_train = X_train.drop(['ScheduledArrival', 'ScheduledDeparture'], axis=1, errors='ignore')
X_test = X_test.drop(['ScheduledArrival', 'ScheduledDeparture'], axis=1, errors='ignore')

# Now you should be able to predict without errors
y_pred = rf_model.predict(X_test)

# Evaluate predictions
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))